In [49]:
from kafka import  KafkaProducer
import sys, os, json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os
import sys
from pyspark.sql.functions import *
from pyspark.sql.functions import to_json, struct


In [50]:
import kafka

In [37]:
topic_name = 'quickstart-events4'
bootstrap_server="localhost:9092"

In [38]:
scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.3.0'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}')

Found existing args: --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.apache.kafka:kafka-clients:3.5.1 pyspark-shell


In [39]:
# set Java home
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_251"

In [40]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","c:/pyspark/*")
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

In [42]:
uid="chaithanya"
pwd="12345678"

#sql db details
server = "localhost"
src_db = "prac"
src_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [28]:
src_url = f"jdbc:sqlserver://{server}:1433;databaseName={src_db};user={uid};password={pwd};"

In [29]:
query="""select emp_id,emp_name,salary,manager_id from prac.dbo.emp_manager"""
table="emp_manager"

In [43]:
dfs=etl.read. \
    format("jdbc"). \
    options(driver=src_driver, user=uid, password=pwd, url=src_url,query=query). \
    load()
dfs.show()

+------+--------+------+----------+
|emp_id|emp_name|salary|manager_id|
+------+--------+------+----------+
|     1|   Ankit| 10000|         4|
|     2|   Mohit| 15000|         5|
|     3|   Vikas| 10000|         4|
|     4|   Rohit|  5000|         2|
|     5|   Mudit| 12000|         6|
|     6|    Agam| 12000|         2|
|     7|  Sanjay|  9000|         2|
|     8|  Ashish|  5000|         2|
+------+--------+------+----------+



Exception while connecting Kafka Unrecognized configs: {'bootstrap_server': ['localhost:9092']}
Unrecognized configs: {'bootstrap_server': ['localhost:9092']}


AttributeError: 'DataFrame' object has no attribute 'to_json'

In [47]:
kafka_df = dfs.selectExpr("CAST(emp_id AS STRING) AS key", "to_json(struct(*)) AS value")
kafka_df.show()



+---+--------------------+
|key|               value|
+---+--------------------+
|  1|{"emp_id":1,"emp_...|
|  2|{"emp_id":2,"emp_...|
|  3|{"emp_id":3,"emp_...|
|  4|{"emp_id":4,"emp_...|
|  5|{"emp_id":5,"emp_...|
|  6|{"emp_id":6,"emp_...|
|  7|{"emp_id":7,"emp_...|
|  8|{"emp_id":8,"emp_...|
+---+--------------------+



In [48]:
kafka_df \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_server) \
    .option("topic", topic_name) \
    .save()
print("Data written to Kafka topic 'Start'")


Data written to Kafka topic 'Start'
